In [37]:
# Problems 8-10 on https://work.caltech.edu/homework/hw7.pdf
import numpy as np
from sklearn import svm
import random


# TODO: adapt perceptron from perceptron.ipynb
# use scikit-learn for svm


def runs(n, run_count=1000):
    total_svm_win = 0
    total_support_vectors = 0
    for i in range(0, run_count):
        svm_win, support_vectors = run(n)
        total_svm_win += svm_win
        total_support_vectors += support_vectors
    
    svm_win_rate = total_svm_win / run_count
    avg_support_vectors = total_support_vectors / run_count

    return svm_win_rate, avg_support_vectors
    
def run(n):
    slope, intercept = generate_line()
    xs = np.random.uniform(-1, 1, (n, 2))
    line_heights = (slope * xs[:, 0]) + intercept
    ys = np.where(xs[:, 1] > line_heights, 1, -1)

    # Restart the run if all elements are the same
    if len(set(ys)) == 1:
        return run(n)

    xs_with_ones = np.hstack((np.ones((n,1)), xs)) # Set x_0 = 1 for all xs
    
    pla_weights = np.zeros(3)

    while True:
        # Predict y's with current weights, and note which ones are misclassified
        predicted_ys = np.sign(xs_with_ones @ pla_weights)
        misclassified_points = np.flatnonzero(predicted_ys != ys)

        # break if no more classified points
        if len(misclassified_points) == 0:
            break

        # Pick a random misclassified point
        misclassified_i = random.choice(misclassified_points)
        x = xs_with_ones[misclassified_i]
        y = ys[misclassified_i]
        # update weights
        pla_weights = pla_weights + (y * x)
    
    # SVM

    clf = svm.SVC(kernel='linear', C=99999999999) # They won't let us actually pass Inf
    clf.fit(xs, ys)

    support_vectors = len(clf.support_vectors_)

    # Generate 30 random points
    TEST_COUNT = 30
    xs = np.random.uniform(-1, 1, (TEST_COUNT, 2))
    line_heights = (slope * xs[:, 0]) + intercept
    ys = np.where(xs[:, 1] > line_heights, 1, -1)

    # Get PLA accuracy
    xs_with_ones = np.hstack((np.ones((30,1)), xs)) # set x_0's = 1
    predicted_ys = np.sign(xs_with_ones @ pla_weights)
    pla_correct_count = len(np.flatnonzero(predicted_ys == ys))
    pla_accuracy = pla_correct_count / len(ys)

    # Get SVM accuracy
    svm_accuracy = clf.score(xs, ys)

    svm_win = svm_accuracy > pla_accuracy

    return (svm_win, support_vectors)

def generate_line():
    point_1 = generate_point()
    point_2 = generate_point()
    return slope_and_intercept(point_1, point_2)

def slope_and_intercept(point_1, point_2):
    slope = (point_2[1] - point_1[1]) / (point_2[0] - point_1[0])
    intercept = point_1[1] - (slope * point_1[0])
    return slope, intercept


def generate_point():
    return np.random.uniform(-1,1,2)

print(runs(100, 1000))


(0.158, 2.997)


In [10]:
# This code cell is for getting acquainted with SVC for the main code above..

import numpy as np
from sklearn import svm

xs = np.array([[ 1., 1.], [2., 2.], [-1., -1.]])
ys = np.array([1, 1, -1])

clf = svm.SVC(kernel='linear', C=99999999999) # They won't let us actually pass Inf
clf.fit(xs, ys)

print(clf.coef_)
print(clf.intercept_)

print(clf.support_vectors_)
print(clf.n_support_)

# clf.score(X_test, Y_test) 

#TODO: have it show weights

[[0.5 0.5]]
[-0.]
[[-1. -1.]
 [ 1.  1.]]
[1 1]
